In [ ]:
# Run and train model from here

from sagemaker.pytorch import PyTorch
import sagemaker
import time
from Runtime import *

Runtime.bucket_name = 'machine-learning'
Runtime.run_datetime = time.strftime("%Y-%m-%d-%H-%M-%S", time.gmtime())
Runtime.job_name = f'{Runtime.run_datetime}-training'

Runtime.model_name = 'pix2pix-model'

Runtime.dataset_name = 'paired-accuracy-speed-modified-v1-dataset'
#Runtime.dataset_name = 'accuracy-speed-modified-v2-dataset'
#Runtime.dataset_name = 'accuracy-speed-modified-v3-dataset'
#Runtime.dataset_name = '2022-05-09-14-39-18-dataset'
#Runtime.dataset_name = 'U12239-2022-05-11-15-21-19-dataset'
#Runtime.dataset_name = 'day2dusk-v3-dataset'

Runtime.dataset_key = f'datasets/processed-data/{Runtime.dataset_name}/'
#Runtime.dataset_key = f'datasets/processed-data/{Runtime.dataset_name}/match-raws-finals/U12239/'

Runtime.dataset_path = f's3://{Runtime.bucket_name}/datasets/processed-data/{Runtime.dataset_name}/'
#Runtime.dataset_path = f's3://{Runtime.bucket_name}/datasets/processed-data/{Runtime.dataset_name}/match-raws-finals/U12239/'

#Runtime.output_path = f's3://{Runtime.bucket}/training-jobs/{Runtime.model_name}/{Runtime.dataset_name}'
Runtime.output_path = f's3://{Runtime.bucket_name}/training-jobs/{Runtime.model_name}/{Runtime.dataset_name}'

ld_model = 'pix2pix-model'
ld_dataset = '2022-05-09-14-39-18-dataset'
ld_job_name = '2022-07-19-02-33-36-training'
ld_epoch = '20'



env = {
    'SAGEMAKER_REQUIREMENTS': 'requirements.txt' # path relative to `source_dir` below.
}

role = sagemaker.get_execution_role()

pytorch_estimator = PyTorch(entry_point = 'main_original.py', #entry point into the program
                            source_dir='./',
                            output_path=Runtime.output_path,
                            role = role,
                            instance_type='ml.g4dn.4xlarge',
                            instance_count=1,
                            framework_version='1.8.0',
                            env=env,
                            code_location = Runtime.output_path,
                            hyperparameters = {"runtime_var": {'model_name':Runtime.model_name,
                                                            'dataset_name':Runtime.dataset_name,
                                                            'job_name':Runtime.job_name,
                                                              'dataset_key':Runtime.dataset_key},

                                               "load_var": {'load_model_name':ld_model,
                                                            'load_dataset_name':ld_dataset,
                                                            'load_job_name':ld_job_name,
                                                            'load_epoch':ld_epoch}
                                               },
                            #use_spot_instances = True,
                            max_run = 48*60*60,
                            #max_wait = 50*60*60,
                            py_version='py3')

pytorch_estimator.fit({'train': Runtime.dataset_path}, job_name=Runtime.job_name, wait=False)
